In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time

import warnings
warnings.filterwarnings('ignore')

In [2]:
starting_dir = os.getcwd()
print ("starting_dir:" + str(starting_dir))

# mac - phase I & II - morpho
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/morpho/tall/2_after_fixing_MO24/final_full/zf_morphology_data_335_chemicals_2020DEC16_fixed.csv'
# 335 chemicals

# mac - phase III - morpho - 23 endpoints - full
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_23_endpoints.csv'

# mac - phase III - morpho - 23 endpoints - devel
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/devel/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_devel_w_23_endpoints.csv'

# mac - phase III - morpho - 14 endpoints - full
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_14_endpoints.csv'

# mac - phase III - morpho - 14 endpoints - devel
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/devel/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_devel_w_14_endpoints.csv'

morph_data = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)

morph_data.head()

starting_dir:/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/morpho


,chemical.id,bottle.id,conc,plate.id,well,date,endpoint,value
0,1030,C48962,0.0,9414,A06,20151028,MO24,0.0
1,1030,C48962,0.0,9414,B06,20151028,MO24,0.0
2,1030,C48962,0.0,9414,C06,20151028,MO24,0.0
3,1030,C48962,0.0,9414,D06,20151028,MO24,0.0
4,1030,C48962,0.0,9414,E06,20151028,MO24,0.0


In [70]:
# just TX part
#TX_bottle_id_morph_data = morph_data[morph_data['bottle.id'].str.contains("^TX")]
#TX_bottle_id_morph_data = morph_data[morph_data['bottle.id'].str.contains("TX$")]
#TX_bottle_id_morph_data.head()

In [71]:
# basic analysis
#new_morph_data = morph_data.loc[morph_data['chemical.id'] == 197,['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#new_morph_data.head()
#morph_data_w_end = new_morph_data.loc[new_morph_data['endpoint'] == 'MO24',['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#morph_data_w_end.head()
#np.sum(morph_data['value'] == 1)
#np.sum(morph_data['value'] == 0) # -> 82%
#np.sum(np.isnan(morph_data['value']))
#morph_data.shape

In [3]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'bottle.id', 'plate.id', 'well', 'endpoint', 'value'] 
# 'bottle.id' is added since Lisa wants to use it to join with behavioral data

morph_data_select = morph_data.loc[:,columns_to_keep]
morph_data_select.head()
morph_data_select.tail()

,chemical.id,conc,bottle.id,plate.id,well,endpoint,value
1600795,998,67.0,NTP DNT 91-B10,15822,A08,DNC_,0.0
1600796,998,67.0,NTP DNT 91-B10,15822,A09,DNC_,0.0
1600797,998,67.0,NTP DNT 91-B10,15822,A10,DNC_,0.0
1600798,998,67.0,NTP DNT 91-B10,15822,A11,DNC_,0.0
1600799,998,67.0,NTP DNT 91-B10,15822,A12,DNC_,0.0


In [4]:
print ("<before dropping duplicates> len(morph_data_select):" + str(len(morph_data_select)))
morph_data_select = morph_data_select.drop_duplicates()
print ("<after dropping duplicates>  len(morph_data_select):" + str(len(morph_data_select)))

<before dropping duplicates> len(morph_data_select):1600800
<after dropping duplicates>  len(morph_data_select):1600800


In [5]:
unique_chemical_id = np.unique(morph_data_select['chemical.id'])
print(len(unique_chemical_id))

335


In [ ]:
#skip
#'''
#unique_endpoints = np.unique(morph_data_select['endpoint'])
#print(unique_endpoints)
#print(len(unique_endpoints))
#'''

In [137]:
#morph_this_chem_only = morph_data_select.loc[morph_data['chemical.id'] == 1694,:]
#morph_this_chem_only.to_csv('1694.csv', index=False)

In [6]:
# tall -> wide
start_time = time.time()

reformatted = pd.DataFrame()

#full_devel = "full"
full_devel = "devel"

chemical_id_from_here = np.unique(morph_data_select['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

if (os.path.isfile("cannot_process.txt") == True):
    os.remove("cannot_process.txt")
        
for chemical_index in chemical_id_from_here:
    print("\nchemical_index:" + str(chemical_index))        
    
    morph_data_chemical = morph_data_select.loc[morph_data['chemical.id'] == chemical_index,:]
    #morph_data_chemical.to_csv('morph_data_chemical_1694.csv', index=False)
#    display(morph_data_chemical.head())
    
    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))

    #print ("len(morph_data_chemical.chemical_plate_well):" + str(len(morph_data_chemical.chemical_plate_well)))    
    for cpw in np.unique(morph_data_chemical.chemical_plate_well):
        df_per_cpw = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]
        #df_per_cpw_filename = 'df_per_cpw_' + str(chemical_index) + '.csv'
        #df_per_cpw.to_csv(df_per_cpw_filename, index=False)
        
        if (len(np.unique(df_per_cpw['conc'])) != 1):
            print ("can't process" + str(chemical_index) + " chemical")
            f = open("cannot_process.txt", "a")
            f.write(str(chemical_index)+"\n")
            f.close()
            break
        
        try:
            plate_id = pd.to_numeric(df_per_cpw['plate.id'])
        except:
            # we need this to deal exceptional case like "plate.id = TP967-E9-P1 for chemical 163 in phase I,II"
            plate_id = df_per_cpw['plate.id']

        if(len(df_per_cpw.endpoint) == 14):
            concat_this = pd.DataFrame(
                {
                    'chemical.id': np.unique(df_per_cpw['chemical.id']),
                    'plate.id': np.unique(plate_id),
                    'well': np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc': np.unique(df_per_cpw['conc']),
                    'AXIS': df_per_cpw.value[df_per_cpw.endpoint == 'AXIS'].values,
                    'BRN_': df_per_cpw.value[df_per_cpw.endpoint == 'BRN_'].values,
                    'CRAN': df_per_cpw.value[df_per_cpw.endpoint == 'CRAN'].values,
                    'DNC_': df_per_cpw.value[df_per_cpw.endpoint == 'DNC_'].values,
                    'DP24': df_per_cpw.value[df_per_cpw.endpoint == 'DP24'].values,
                    'EDEM': df_per_cpw.value[df_per_cpw.endpoint == 'EDEM'].values,
                    'LTRK': df_per_cpw.value[df_per_cpw.endpoint == 'LTRK'].values,
                    'MO24': df_per_cpw.value[df_per_cpw.endpoint == 'MO24'].values,
                    'MORT': df_per_cpw.value[df_per_cpw.endpoint == 'MORT'].values,
                    'MUSC': df_per_cpw.value[df_per_cpw.endpoint == 'MUSC'].values,
                    'NC__': df_per_cpw.value[df_per_cpw.endpoint == 'NC__'].values,
                    'SKIN': df_per_cpw.value[df_per_cpw.endpoint == 'SKIN'].values,
                    'SM24': df_per_cpw.value[df_per_cpw.endpoint == 'SM24'].values,
                    'TCHR': df_per_cpw.value[df_per_cpw.endpoint == 'TCHR'].values,
                })
            reformatted = pd.concat([reformatted, concat_this])
        elif(len(df_per_cpw.endpoint) == 23):
            concat_this = pd.DataFrame(
                {
                    'chemical.id':         np.unique(df_per_cpw['chemical.id']),
                    'plate.id':            np.unique(plate_id),
                    'well':                np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc':                np.unique(df_per_cpw['conc']),
                    'AXIS': df_per_cpw.value[df_per_cpw.endpoint == 'AXIS'].values,
                    'BRAI': df_per_cpw.value[df_per_cpw.endpoint == 'BRAI'].values,
                    'CFIN': df_per_cpw.value[df_per_cpw.endpoint == 'CFIN'].values,
                    'CIRC': df_per_cpw.value[df_per_cpw.endpoint == 'CIRC'].values,
                    'DNC_': df_per_cpw.value[df_per_cpw.endpoint == 'DNC_'].values,
                    'DP24': df_per_cpw.value[df_per_cpw.endpoint == 'DP24'].values,
                    'EYE_': df_per_cpw.value[df_per_cpw.endpoint == 'EYE_'].values,
                    'JAW_': df_per_cpw.value[df_per_cpw.endpoint == 'JAW_'].values,
                    'MO24': df_per_cpw.value[df_per_cpw.endpoint == 'MO24'].values,
                    'MORT': df_per_cpw.value[df_per_cpw.endpoint == 'MORT'].values,
                    'NC24': df_per_cpw.value[df_per_cpw.endpoint == 'NC24'].values,
                    'NC__': df_per_cpw.value[df_per_cpw.endpoint == 'NC__'].values,
                    'OTIC': df_per_cpw.value[df_per_cpw.endpoint == 'OTIC'].values,
                    'PE__': df_per_cpw.value[df_per_cpw.endpoint == 'PE__'].values,
                    'PFIN': df_per_cpw.value[df_per_cpw.endpoint == 'PFIN'].values,
                    'PIG_': df_per_cpw.value[df_per_cpw.endpoint == 'PIG_'].values,
                    'SM24': df_per_cpw.value[df_per_cpw.endpoint == 'SM24'].values,
                    'SNOU': df_per_cpw.value[df_per_cpw.endpoint == 'SNOU'].values,
                    'SOMI': df_per_cpw.value[df_per_cpw.endpoint == 'SOMI'].values,
                    'SWIM': df_per_cpw.value[df_per_cpw.endpoint == 'SWIM'].values,
                    'TRUN': df_per_cpw.value[df_per_cpw.endpoint == 'TRUN'].values,
                    'TR__': df_per_cpw.value[df_per_cpw.endpoint == 'TR__'].values,
                    'YSE_': df_per_cpw.value[df_per_cpw.endpoint == 'YSE_'].values,
                })
            reformatted = pd.concat([reformatted, concat_this])
        elif(len(df_per_cpw.endpoint) == 30):
            concat_this = pd.DataFrame(
                {
                    'chemical.id': np.unique(df_per_cpw['chemical.id']),
                    'plate.id': np.unique(plate_id),
                    'well': np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc': np.unique(df_per_cpw['conc']),
                    'AXIS': df_per_cpw.value[df_per_cpw.endpoint == 'AXIS'].values,
                    'BRAI': df_per_cpw.value[df_per_cpw.endpoint == 'BRAI'].values,
                    'BRN_': df_per_cpw.value[df_per_cpw.endpoint == 'BRN_'].values,
                    'CFIN': df_per_cpw.value[df_per_cpw.endpoint == 'CFIN'].values,
                    'CIRC': df_per_cpw.value[df_per_cpw.endpoint == 'CIRC'].values,
                    'CRAN': df_per_cpw.value[df_per_cpw.endpoint == 'CRAN'].values,
                    'DNC_': df_per_cpw.value[df_per_cpw.endpoint == 'DNC_'].values,
                    'DP24': df_per_cpw.value[df_per_cpw.endpoint == 'DP24'].values,
                    'EDEM': df_per_cpw.value[df_per_cpw.endpoint == 'EDEM'].values,
                    'EYE_': df_per_cpw.value[df_per_cpw.endpoint == 'EYE_'].values,
                    'JAW_': df_per_cpw.value[df_per_cpw.endpoint == 'JAW_'].values,
                    'LTRK': df_per_cpw.value[df_per_cpw.endpoint == 'LTRK'].values,
                    'MO24': df_per_cpw.value[df_per_cpw.endpoint == 'MO24'].values,
                    'MORT': df_per_cpw.value[df_per_cpw.endpoint == 'MORT'].values,
                    'MUSC': df_per_cpw.value[df_per_cpw.endpoint == 'MUSC'].values,
                    'NC24': df_per_cpw.value[df_per_cpw.endpoint == 'NC24'].values,
                    'NC__': df_per_cpw.value[df_per_cpw.endpoint == 'NC__'].values,
                    'OTIC': df_per_cpw.value[df_per_cpw.endpoint == 'OTIC'].values,
                    'PE__': df_per_cpw.value[df_per_cpw.endpoint == 'PE__'].values,
                    'PFIN': df_per_cpw.value[df_per_cpw.endpoint == 'PFIN'].values,
                    'PIG_': df_per_cpw.value[df_per_cpw.endpoint == 'PIG_'].values,
                    'SM24': df_per_cpw.value[df_per_cpw.endpoint == 'SM24'].values,
                    'SNOU': df_per_cpw.value[df_per_cpw.endpoint == 'SNOU'].values,
                    'SOMI': df_per_cpw.value[df_per_cpw.endpoint == 'SOMI'].values,
                    'SWIM': df_per_cpw.value[df_per_cpw.endpoint == 'SWIM'].values,
                    'TCHR': df_per_cpw.value[df_per_cpw.endpoint == 'TCHR'].values,
                    'TRUN': df_per_cpw.value[df_per_cpw.endpoint == 'TRUN'].values,
                    'TR__': df_per_cpw.value[df_per_cpw.endpoint == 'TR__'].values,
                    'YSE_': df_per_cpw.value[df_per_cpw.endpoint == 'YSE_'].values,
                })
            reformatted = pd.concat([reformatted, concat_this])
        else:
            print ("len(group.endpoint):" + str(len(group.endpoint)))
            a=b
                  
    
################## possible manual update of entries
find_this = reformatted.loc[(reformatted['chemical.id'] == 746) \
                & (reformatted['plate.id'] == 12953) \
                & (reformatted['well'] == "E03"), 'MORT'] 
if (len(find_this) != 0):
    print ("for phase I, II, automatically fix small manual mistakes")
    reformatted.loc[(reformatted['chemical.id'] == 746) \
                & (reformatted['plate.id'] == 12953) \
                & (reformatted['well'] == "E03"), 'MORT'] = 0

    
find_this = reformatted.loc[(reformatted['chemical.id'] == 1171) \
                    & (reformatted['plate.id'] == 15205) \
                    & (reformatted['well'] == "D02"), 'MORT']
if (len(find_this) != 0):
    print ("for phase I, II, automatically fix small manual mistakes")
    reformatted.loc[(reformatted['chemical.id'] == 1171) \
                    & (reformatted['plate.id'] == 15205) \
                    & (reformatted['well'] == "D02"), 'MORT'] = 0

       
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("tall -> wide done, it took:"+str(time_took)) # took 13 minutes for 344 chemicals

SyntaxError: invalid syntax (<ipython-input-6-de9e1d949746>, line 18)

In [77]:
'''endpoint_investigate_data = pd.read_csv(endpoint_report_filename, header = 0)

print ("<before dropping duplicates> len(endpoint_investigate_data):" + str(len(endpoint_investigate_data)))
unique_endpoint_investigate_data = endpoint_investigate_data.drop_duplicates()
print ("<after dropping duplicates>  len(unique_endpoint_investigate_data):" + str(len(unique_endpoint_investigate_data)))

#display(unique_endpoint_investigate_data)

#unique_endpoint_investigate_data.to_csv('unique_endpoint_investigate_data.csv', index=False)
'''

'endpoint_investigate_data = pd.read_csv(endpoint_report_filename, header = 0)\n\nprint ("<before dropping duplicates> len(endpoint_investigate_data):" + str(len(endpoint_investigate_data)))\nunique_endpoint_investigate_data = endpoint_investigate_data.drop_duplicates()\nprint ("<after dropping duplicates>  len(unique_endpoint_investigate_data):" + str(len(unique_endpoint_investigate_data)))\n\n#display(unique_endpoint_investigate_data)\n\n#unique_endpoint_investigate_data.to_csv(\'unique_endpoint_investigate_data.csv\', index=False)\n'

In [78]:
#print (reformatted.head())
#print ("reformatted.shape:" + str(reformatted.shape))

In [138]:
reformatted_DNC_0 = pd.DataFrame()
reformatted_DNC_0 = reformatted.loc[reformatted['DNC_'] == 0.0]
print ("reformatted_DNC_0.shape:" + str(reformatted_DNC_0.shape))

reformatted_DNC_0.shape:(65006, 28)


In [52]:
reformatted_DNC_empty = pd.DataFrame()
reformatted_DNC_empty = reformatted.loc[reformatted['DNC_'] == '']
print ("reformatted_DNC_empty.shape:" + str(reformatted_DNC_empty.shape))

reformatted_DNC_empty.shape:(0, 28)


In [81]:
#'''
nan_count = 0
zero_count = 0
one_count = 0
total_count = 0
for (columnName, columnData) in reformatted.iteritems(): 
    if (columnName == "chemical.id") or (columnName == "plate.id") or (columnName == "well") or (columnName == "chemical_plate_well") or (columnName == "conc"):
        continue
#    print('Colunm Name : ', columnName) 
#    print('Column Contents : ', columnData.values[i])
#    print (len(columnData.values))
    for i in range(len(columnData.values)):
        total_count += 1
        if (str(columnData.values[i]) == "nan"):
            nan_count += 1
        elif (str(columnData.values[i]) == "0.0"):
            zero_count += 1
        elif (str(columnData.values[i]) == "1.0"):
            one_count += 1
#        print('Column Contents : ', columnData.values[i])
print ("nan_count:" + str(nan_count))
print ("zero_count:" + str(zero_count))
print ("one_count:" + str(one_count))
print ("total_count:" + str(total_count))
#'''

nan_count:123408
zero_count:730792
one_count:22376
total_count:876576


In [82]:
#pd.set_option('display.max_columns', None)

In [139]:
#reformatted.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_all.csv',index=False)

output_complete_file_path = complete_file_path[:-4] + "_wide_DNC_0_" \
                            + str(full_devel) + ".csv"
display ('output_complete_file_path:', str(output_complete_file_path))

reformatted_DNC_0.to_csv(output_complete_file_path,index=False)

#reformatted_DNC_empty.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_empty.csv',index=False)

'output_complete_file_path:'

'/Users/kimd999/research/projects/toxicity/per_each_data/Phase_I_II/input/morpho/tall/zf_morphology_data_335_chemicals_2020DEC16_wide_DNC_0_full.csv'